[example apis](https://www.here.com/docs/bundle/data-sdk-for-python-developer-guide-v2/page/topics/notebooks.html)

In [1]:
import requests, time

api_key = "xrcYx5euwWciYdK-coeSXcu16kjdJAujw9INxZRAp_U"

# "Profile mode (car fastest)" example request
request = {
    "origins": [{"lat": 52.5309, "lng": 13.3849}, {"lat": 54.0924, "lng": 12.0991}],
    "destinations": [
        {"lat": 51.3397, "lng": 12.3731},
        {"lat": 51.0504, "lng": 13.7373},
    ],
    "regionDefinition": {"type": "world"},
    "profile": "carFast",
}

In [2]:


# Using a Session object allows you to persist certain parameters accross requests
# see: https://requests.readthedocs.io/en/master/user/advanced/
session = requests.Session()
# Add `?apiKey=xxxxxx` to all requests
session.params = {"apiKey": api_key}
# Raise an exception for any non 2xx or 3xx HTTP return codes
session.hooks = {"response": lambda r, *args, **kwargs: r.raise_for_status()}

# Send an asynchronous request, see: https://developer.here.com/documentation/matrix-routing-api/8.3.0/dev_guide/topics/get-started/asynchronous-request.html
status_response = session.post(
    "https://matrix.router.hereapi.com/v8/matrix", json=request
).json()

# Polling for the job status for 100 times
# You might want to use exponential back-off instead of time.sleep
for _ in range(0, 100):
    # do not follow the redirect here
    status_response = session.get(
        status_response["statusUrl"], allow_redirects=False
    ).json()

    if status_response["status"] == "completed":
        # download the result
        matrix_response = session.get(status_response["resultUrl"]).json()
        print(matrix_response)
        break
    elif status_response["accepted"] or status_response["inProgress"]:
        continue
    else:
        print(f"An error occured: {status_response}")
        break
    time.sleep(0.5)  # sleep for 500 ms

SSLError: HTTPSConnectionPool(host='matrix.router.hereapi.com', port=443): Max retries exceeded with url: /v8/matrix?apiKey=xrcYx5euwWciYdK-coeSXcu16kjdJAujw9INxZRAp_U (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1006)')))

In [16]:
import os
import json
from path import Path
import requests
import certifi
from here_location_services.exceptions import ConfigException
from here_map_widget import Map, GeoJSON
import urllib3
urllib3.disable_warnings()

## try with auoth code

In [3]:
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
    pass
else:
    # Handle target environment that doesn't support HTTPS verification
    ssl._create_default_https_context = _create_unverified_https_context

In [21]:
from here_location_services import LS
from here_location_services import PlatformCredentials


# platform_credentials = PlatformCredentials.from_env()
platform_credentials = PlatformCredentials.from_credentials_file(r"C:\Users\nanzawi\OneDrive - HDR, Inc\Data_Science\MODOT_RR\HERE_APIs\.here\credentials.properties")

ls = LS(platform_credentials=platform_credentials)


SSLError: HTTPSConnectionPool(host='account.api.here.com', port=443): Max retries exceeded with url: /oauth2/token (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1006)')))

### Try with API key

In [9]:
HERE_USER_ID = "HERE-21a88e98-cb2a-437c-95e4-045a418a72af"
HERE_CLIENT_ID = "vL9srpcDJwofq0vTpFQy"
HERE_ACCESS_KEY_ID = "ZvObrUoexEwKC3KgiMuy3w"
HERE_ACCESS_KEY_SECRET = "mkupOBOzTREwAaqFwIsr9Z0Cmax8FV_YRKYiDY1SBlZ1FYfjI7QOtcCcy4D-d2rpdlxrncRAHFwvp-Zu4wgClg"
HERE_TOKEN_ENDPOINT_URL = "https://account.api.here.com/oauth2/token"

In [10]:
LS_API_KEY = os.environ.get("coeSXcu16kjdJAujw9INxZRAp_U")  # Get API KEY from environment.
ls = LS(api_key=LS_API_KEY)

ConfigException: Missing environmental variables: HERE_USER_ID, HERE_CLIENT_ID, HERE_ACCESS_KEY_ID, HERE_ACCESS_KEY_SECRET

### try with api

In [3]:
import time #To generate the OAuth timestamp
import urllib.parse #To URLencode the parameter string
import hmac #To implement HMAC algorithm
import hashlib #To generate SHA256 digest
from base64 import b64encode #To encode binary data into Base64
import binascii #To convert data into ASCII
import requests #To make HTTP requests

In [4]:
grant_type = 'client_credentials'
oauth_consumer_key = 'ZvObrUoexEwKC3KgiMuy3w' #From credentials.properties file
oauth_nonce = str(int(time.time()*1000))
oauth_signature_method = 'HMAC-SHA256'
oauth_timestamp = str(int(time.time()))
oauth_version = '1.0'

In [5]:
def create_parameter_string(grant_type, oauth_consumer_key,oauth_nonce,oauth_signature_method,oauth_timestamp,oauth_version):
    parameter_string = ''
    parameter_string = parameter_string + 'grant_type=' + grant_type
    parameter_string = parameter_string + '&oauth_consumer_key=' + oauth_consumer_key
    parameter_string = parameter_string + '&oauth_nonce=' + oauth_nonce
    parameter_string = parameter_string + '&oauth_signature_method=' + oauth_signature_method
    parameter_string = parameter_string + '&oauth_timestamp=' + oauth_timestamp
    parameter_string = parameter_string + '&oauth_version=' + oauth_version
    return parameter_string

parameter_string = create_parameter_string(grant_type, oauth_consumer_key,oauth_nonce,oauth_signature_method,oauth_timestamp,oauth_version)
encoded_parameter_string = urllib.parse.quote(parameter_string, safe='')

In [6]:
url = 'https://account.api.here.com/oauth2/token'
encoded_base_string = 'POST' + '&' + urllib.parse.quote(url, safe='')
encoded_base_string = encoded_base_string + '&' + encoded_parameter_string

In [7]:
access_key_secret = 'mkupOBOzTREwAaqFwIsr9Z0Cmax8FV_YRKYiDY1SBlZ1FYfjI7QOtcCcy4D-d2rpdlxrncRAHFwvp-Zu4wgClg' #From credentials.properties file
signing_key = access_key_secret + '&'

def create_signature(secret_key, signature_base_string):
    encoded_string = signature_base_string.encode()
    encoded_key = secret_key.encode()
    temp = hmac.new(encoded_key, encoded_string, hashlib.sha256).hexdigest()
    byte_array = b64encode(binascii.unhexlify(temp))
    return byte_array.decode()

oauth_signature = create_signature(signing_key, encoded_base_string)
encoded_oauth_signature = urllib.parse.quote(oauth_signature, safe='')

In [8]:
body = {'grant_type' : '{}'.format(grant_type)}

headers = {
            'Content-Type' : 'application/x-www-form-urlencoded',
            'Authorization' : 'OAuth oauth_consumer_key="{0}",oauth_nonce="{1}",oauth_signature="{2}",oauth_signature_method="HMAC-SHA256",oauth_timestamp="{3}",oauth_version="1.0"'.format(oauth_consumer_key,oauth_nonce,encoded_oauth_signature,oauth_timestamp)
          }
    
response = requests.post(url, data=body, headers=headers, verify = False)

print(response.text)

C:\Users\nanzawi\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'account.api.here.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{"access_token":"eyJhbGciOiJSUzUxMiIsImN0eSI6IkpXVCIsImlzcyI6IkhFUkUiLCJhaWQiOiJ2TDlzcnBjREp3b2ZxMHZUcEZReSIsImlhdCI6MTY5ODA3OTIxMSwiZXhwIjoxNjk4MTY1NjExLCJraWQiOiJqMSJ9.ZXlKaGJHY2lPaUprYVhJaUxDSmxibU1pT2lKQk1qVTJRMEpETFVoVE5URXlJbjAuLmVROGQxMjVxTFlRamxHcWJJbFpZZGcuREV5eDdpdW9VR0ZveG1sX1Z6QWtYRzFnNnh2dEpjbTRZTFlTSWVhMncwakE1NjNkcnJjV2hyM2NtOElWTkRWNjU1RVBZa3h2dkZNalgyc0VhRUFXclNOVWRmVFlxd0dqUlVlb0dwd3NvdC1mOTdnZi01bkJLaDRwY3ItWW1jaFFNWnNNbmwwWHRNVlcycExnTnpQX3BIQ3dNamdyUlBDVDFjTnFCbkN5QVdVLl9SdmFZY0RxZ3U2TllseTNKQ3Nsc1BsaUtWYXRIUWFZSnF0cEhKOXR0SzA.vAlXKjfXzGITu66NflZ_GCGu0HZwrLXzKziXugy8zDLSdVZEWF2I64qIhT6M2xS2R3a5myboqFm7umi0krV5a_BcO6h4UcvHzds6nLLmPZDVyEzFPxXdnouNTJS0gD_Xy2ZApY1W4g5Q48YbUJys06SjB25VDQGCy_I1fU82nOzPpgv9ejlyYH-LSA4GstqfASNba4IUuxKdXo4R1p9hKuOJTF7pUGLAIBodoc7FXvBcXVf9Df9MUHFnz4oQDxgb-urg952240cDylup3EjAKCFU8fu2pm_xJwFk8ZTwM3JkJ_BzTBEbWcT4_QOQ1V_WSunHSe14a-YA9qOgBADmkQ","token_type":"bearer","expires_in":86399}


In [38]:
YOUR_API_KEY= 'xrcYx5euwWciYdK-coeSXcu16kjdJAujw9INxZRAp_U'
url = f'https://router.hereapi.com/v8/routes?transportMode=car&origin=52.5308,13.3847&destination=52.5264,13.3686&return=summary&apikey={YOUR_API_KEY}'

response = requests.get(url,verify=False)
data = response.json()

In [40]:
origin = "52.5200,13.4050"  # Example: Berlin, Germany
destination = "48.8566,2.3522"  # Example: Paris, France

In [48]:
# Define a function to call a routing API
def call_here_routing_api(origin, destination):
    # Replace with your actual HERE API credentials
    app_id = "vL9srpcDJwofq0vTpFQy"
    app_code = "coeSXcu16kjdJAujw9INxZRAp_U"

    url = f'https://router.hereapi.com/v8/routes?transportMode=car&origin={origin}&destination={destination}&return=summary&apikey={YOUR_API_KEY}'
    response = requests.get(url, verify = False)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        return None

In [ ]:
result = call_here_routing_api(origin, destination)

In [50]:
result

{'routes': [{'id': 'a6b23a28-1864-4de7-8a37-9927e3393178',
   'sections': [{'id': '2518d51d-7658-45cf-b465-af4c1f1af839',
     'type': 'vehicle',
     'departure': {'time': '2023-10-23T20:17:01+02:00',
      'place': {'type': 'place',
       'location': {'lat': 52.52003, 'lng': 13.40495},
       'originalLocation': {'lat': 52.5199999, 'lng': 13.405}}},
     'arrival': {'time': '2023-10-24T06:28:15+02:00',
      'place': {'type': 'place',
       'location': {'lat': 48.85727, 'lng': 2.35232},
       'originalLocation': {'lat': 48.8566, 'lng': 2.3522}}},
     'summary': {'duration': 36674, 'length': 1053753, 'baseDuration': 35729},
     'transport': {'mode': 'car'}}]}]}